
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [11]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


In [12]:
from ipynb.fs.defs.GradProject_NB2 import feature_frame

In [18]:
data = feature_frame()
y = data['Label']
x = data.drop(columns=['Label', 'Image_Id'])
print("Done preprocessing")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

Fetching label '.DS_Store'
Fetching label 'airplanes'
Fetching label 'bear'
Fetching label 'blimp'
	Gray image ('blimp_0022.jpg') was loaded, converting to RGB
Fetching label 'comet'
	Gray image ('comet_0006.jpg') was loaded, converting to RGB
	Gray image ('comet_0011.jpg') was loaded, converting to RGB
	Gray image ('comet_0013.jpg') was loaded, converting to RGB
	Gray image ('comet_0021.jpg') was loaded, converting to RGB
	Gray image ('comet_0036.jpg') was loaded, converting to RGB
	Gray image ('comet_0038.jpg') was loaded, converting to RGB
	Gray image ('comet_0041.jpg') was loaded, converting to RGB
	Gray image ('comet_0049.jpg') was loaded, converting to RGB
	Gray image ('comet_0052.jpg') was loaded, converting to RGB
	Gray image ('comet_0053.jpg') was loaded, converting to RGB
	Gray image ('comet_0057.jpg') was loaded, converting to RGB
	Gray image ('comet_0058.jpg') was loaded, converting to RGB
Fetching label 'crab'
	Gray image ('crab_0045.jpg') was loaded, converting to RGB
Fet

In [19]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

Logistic Regression

In [43]:
model = LogisticRegression()
model.fit(x_train_scaled, y_train)
preds = model.predict(x_test_scaled)
acc = accuracy_score(preds, y_test)
print(acc)

0.3388704318936877


K-nearest Neighbors

In [55]:
model = KNeighborsClassifier(10)
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

0.2757475083056478


Classification Tree

In [42]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

0.31893687707641194


Random Forest

In [38]:
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train, y_train)
preds = model.predict(x_test)
acc = accuracy_score(preds, y_test)
print(acc)

s = model.feature_importances_
print(s[:10])
index_importance_sorted = sorted(range(len(s)), key=lambda k: s[k], reverse=True)
print(index_importance_sorted[:10])
top_index = index_importance_sorted[:15]
print("Most importance features:")
for index in top_index:
    print(f"Feature name: {x_train.columns[index]}, Importance={s[index]}")

0.39867109634551495
[0.01550203 0.01268453 0.00373825 0.00456659 0.00408338 0.00381922
 0.0039683  0.00388937 0.00082096 0.00022223]
[0, 1, 151, 288, 224, 223, 71, 150, 444, 215]
Most importance features:
Feature name: Size, Importance=0.015502033886434265
Feature name: AspectRatio, Importance=0.012684531797931501
Feature name: hist_feat_145, Importance=0.006982690563935953
Feature name: hist_feat_282, Importance=0.006318578776059586
Feature name: hist_feat_218, Importance=0.006314272346001445
Feature name: hist_feat_217, Importance=0.005806074088000666
Feature name: hist_feat_65, Importance=0.0056557492668981866
Feature name: hist_feat_144, Importance=0.0055705248964820556
Feature name: hist_feat_438, Importance=0.005563263258499672
Feature name: hist_feat_209, Importance=0.005407967988465401
Feature name: hist_feat_365, Importance=0.005281329589033475
Feature name: hist_feat_355, Importance=0.005081727355642734
Feature name: hist_feat_511, Importance=0.005044010950881174
Feature name

Support Vector Machine

In [26]:
model = SVC()
model.fit(x_train_scaled, y_train)
preds = model.predict(x_test_scaled)
acc = accuracy_score(preds, y_test)
print(acc)

0.08970099667774087
